# Flight Control Design and Nonlinear Simulations of Closed-Loop System
### Raktim Bhattacharya
*Professor* 

Aerospace Engineering, Texas A&M University

raktim@tamu.edu, https://isrlab.github.io

**Note**: Plots will appear once the notebook is executed. May not show up in GitHub.

# Initialize

In [2]:
using F16Model 
using PrettyTables, LinearAlgebra, Plots, Convex, MosekTools, DifferentialEquations

F16Model.PrintStateAndControlBounds()

./loading.jl:997
  [7] include
    @ ./Base.jl:418 [inlined]
  [8] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::String)
    @ Base ./loading.jl:1318
  [9] top-level scope
    @ none:1
 [10] eval
    @ ./boot.jl:373 [inlined]
 [11] eval(x::Expr)
    @ Base.MainInclude ./client.jl:453
 [12] top-level scope
    @ none:1
in expression starting at /Users/sherpa/Dropbox/Mac (2)/Documents/GitHub/F16Model.jl/src/F16Model.jl:1
ERROR: LoadError: ArgumentError: Package ChangesOfVariables does not have ChainRulesCore in its dependencies:
- If you have ChangesOfVariables checked out for development and have
  added ChainRulesCore as a dependency but haven't updated your primary
  environment's manifest file, try `Pkg.resolve()`.
- Otherwise you may need to report an issue with ChangesOfVariables
Stacktrace:
 [1] require(into::Module, mod::Symbol

ErrorException: Failed to precompile F16Model [edccaa42-18b2-46d1-80ce-8508c2c4933c] to /Users/sherpa/.julia/compiled/v1.7/F16Model/jl_ydjuSg.

# Basic Operations

In [14]:
# Define state vector
# -------------------
d2r = pi/180;
npos = 0; # ft
epos = 0; # ft
alt = 10000; # should be in between 5000 ft and 100000 ft
phi = 0;   # rad -- Roll
theta = 0; # rad -- Pitch
psi = 0;   # rad -- Yaw
Vt = 300;  # ft/s -- Total velocity
alp = 0;   # rad -- Angle of attack
bet = 0;   # rad -- Side slip angle 
p = 0;     # rad/s -- Roll rate
q = 0;     # rad/s -- Pitch rate
r = 0;     # rad/s -- Yaw rate

x0 = [npos,epos,alt,phi,theta,psi,Vt,alp,bet,p,q,r];

# Define control vector
# ---------------------
T = 9000; # Thrust lbs
dele = 0; # deg elevator angle
dail = 0; # deg aileron angle
drud = 0; # deg rudder angle
dlef = 0; # deg leading edge flap angle
u0 = [T,dele,dail,drud,dlef];

./loading.jl:997
  [7] include
    @ ./Base.jl:418 [inlined]
  [8] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::String)
    @ Base ./loading.jl:1318
  [9] top-level scope
    @ none:1
 [10] eval
    @ ./boot.jl:373 [inlined]
 [11] eval(x::Expr)
    @ Base.MainInclude ./client.jl:453
 [12] top-level scope
    @ none:1
in expression starting at /Users/sherpa/Dropbox/Mac (2)/Documents/GitHub/F16Model.jl/src/F16Model.jl:1


In [15]:
# Evaluate xdot -- inplace implementation -- use this with DifferentialEquations package.
xdot1 = zeros(12);
F16Model.Dynamics!(xdot1,x0,u0); # Does not implement actuator dynamics.

# Evaluate xdot -- returns vector. Use this for linearization of dynamics, etc.
xdot2 = F16Model.Dynamics(x0,u0); #  Does not implement actuator dynamics.

UndefVarError: UndefVarError: F16Model not defined

# Find Equilibrium (or Trim) Condition
For steady-level flight.

In [16]:
# Trim the aircraft for steady-level flight at h0,V0
# xbar = trim state
# ubar = trim control
# status = status of the optimization, status = 0 means optimization found solution and (xbar, ubar) defines a valid trim  point
# prob = data structure from IpOpt.

h0 = 10000; # ft
Vt0 = 500;   # ft/s

# Stead-Level Flight
xbar, ubar, status, prob = F16Model.Trim(h0,Vt0,γ=0, ψdot=0, ϕ=(0,1), ψ=(0,1), β=(0,1), p=(0,1), q=(0,1), r=(0,1));
xdot_trim = F16Model.Dynamics(xbar,ubar); # Should be close to zero.

UndefVarError: UndefVarError: F16Model not defined

# Check Trim Values

In [17]:
# Check Trim Values
xuBounds = F16Model.StateAndControlBounds();
states = ["npos","epos","h","phi","theta","psi","V","alpha","beta","p","q","r"];
statesDer = ["Dnpos","Depos","Dh","Dphi","Dtheta","Dpsi","DV","Dalpha","Ddbeta","Dp","Dq","Dr"];
controls = ["T","ele","ail","rud","lef"];

xmin = vcat([-Inf;-Inf],xuBounds[1:10,1]); xmax = vcat([Inf;Inf], xuBounds[1:10,2]);

data = hcat(states,xdot_trim,xmin,xbar,xmax)
pretty_table(header=["states","xdot","xmin","xbar","xmax"],data);
pretty_table(header=["controls","umin","ubar","umax"],hcat(controls,xuBounds[11:end,1],ubar,xuBounds[11:end,2]));

UndefVarError: UndefVarError: F16Model not defined

# Linear Analysis

In [18]:
# Linearize about trim point (xbar,ubar)
A, B = F16Model.Linearize(xbar,ubar);

# Pull out longitudinal dynamics -- states:(theta,V,alpha,q), control:(T,ele)
ix = [5,7,8,11]; iu = [1,2]; # 4 states, 3 control system.
longiA = A[ix,ix];
longiB = B[ix,iu];

# Show system
pretty_table(header = vcat("xdotLongi",states[ix],controls[iu]),hcat(statesDer[ix],longiA,longiB))

# Eigen values 
ev = eigen(longiA).values;
pretty_table(header=["Eigen values"],ev)

plot(real(ev),imag(ev),seriestype=:scatter,label="Eigen values")

UndefVarError: UndefVarError: F16Model not defined

# Linear Controller Design
Design state feedback controller $$u = Kx,$$ for the system $$\dot{x}(t) = Ax(t) + Bu(t),$$
which minimizes the cost
$$
\int_0^\infty \left(x^TQx + u^TRu\right)dt.
$$
Can be solved as the following convex optimization problem. See pg. 28 in [this document](https://isrlab.github.io/pdfs/aero632/05a%20StateFeedback%20LQR.pdf).

In [19]:
# Design LQR 
Q = Diagonal([1/0.1,1/100,1/0.1,1/200]);
R = Diagonal([1/5000,1/5]);

ns,nu = size(longiB);
Y = Semidefinite(ns,ns);
W = Convex.Variable(nu,ns)

M11 = (longiA*Y + longiB*W) + (longiA*Y + longiB*W)'
M = [M11 Y W';
     Y  -inv(Q) zeros(ns,nu);
     W   zeros(nu,ns) -inv(R)];
prob = maximize(tr(Y),-M in :SDP)

Convex.solve!(prob,Mosek.Optimizer); # Careful with other solvers. Solution accuracy impacts controller performance.
@show prob.status

UndefVarError: UndefVarError: Diagonal not defined

In [20]:
# Closed-loop linear system
K = W.value*inv(Y.value); # Recover controller gain.
Acl = longiA + longiB*K;  # Closed-loop system.

# Eigen values 
ev_cl = eigen(Acl).values;
pretty_table(header=["OL EV","CL EV"],hcat(ev,ev_cl));

plot(real(ev),imag(ev),seriestype=:scatter,label="Eigen values: OL")
plot!(real(ev_cl),imag(ev_cl),seriestype=:scatter,label="Eigen values: CL")

UndefVarError: UndefVarError: W not defined

We see from the above plot that the closed-loop system has improved the low frequency lightly damped modes (known as phugoid mode). This is reflected in faster decay of perturbations as shown next.

# Simulate Linear System with LQR 

In [21]:
# Simulate linear system
dt = 0.01; tfinal = 15;
tGrid = 0:dt:tfinal;

x0 = [0,50,0,0];

xTraj = hcat([exp(Acl*t)*x0 for t in tGrid]...);
xTrajOL = hcat([exp(longiA*t)*x0 for t in tGrid]...);
xp = [plot(tGrid,[xTraj[i,:],xTrajOL[i,:]],label=["CL" "OL"],xlabel="Time(s)",ylabel=states[ix[i]],lw=2) for i in 1:ns]

uTraj = K*xTraj;
up = [plot(tGrid,uTraj[i,:],label=:none,xlabel="Time(s)",ylabel=controls[iu[i]],lw=2) for i in 1:nu]

display(plot(xp...,up...));

UndefVarError: UndefVarError: Acl not defined

# Simulate Nonlinear Closed-Loop System

In [22]:
# Create nonlinear closed-loop system 
function nonlinear_longi_F16_cl!(xdot,x,p,t)
    xpert = x-xbar;
    u = copy(ubar);
    u[iu] = K*xpert[ix];
    F16Model.Dynamics!(xdot,x,u);
end

# Simulate from some initial condition: 50 ft/s perturbation from trim.
nl_x0 =  xbar + [0,0,0,0,0,0,50,0,0,0,0,0]; # IC is offset from trim.
tspan = (0.0,30);
xdot = zeros(12)
prob = ODEProblem(nonlinear_longi_F16_cl!,nl_x0,tspan);
sol = solve(prob);

UndefVarError: UndefVarError: xbar not defined

In [23]:
# Plot the nonlinear solution and compare with linear solution.
xTrajNL = hcat(sol.u...);
xpert = xTrajNL .- xbar;
uNL = K*xpert[ix,:]; 


xpNL = [begin 
        plot(sol.t,sol[ix[i],:].-xbar[ix[i]],label="NLin",lw=2,xlabel="Time(s)",ylabel=states[ix[i]]); 
        plot!(tGrid,xTraj[i,:],label="Lin",lw=2);
     end for i in 1:ns];

upNL = [begin 
     plot(sol.t,uNL[i,:],label="NLin",lw=2,xlabel="Time(s)",ylabel=controls[iu[i]]); 
     plot!(tGrid,uTraj[i,:],label="Lin",lw=2);
  end for i in 1:nu];

display(plot(xpNL...,upNL...));

UndefVarError: UndefVarError: sol not defined

# Large perturbation
Simulate with velocity perturbation equal to 110 ft/s. 

In [24]:
nl_x0 =  xbar + [0,0,0,0,0,0,110,0,0,0,0,0]; # IC is offset from trim.
tspan = (0.0,30);
xdot = zeros(12)
prob = ODEProblem(nonlinear_longi_F16_cl!,nl_x0,tspan);
sol = solve(prob);

xTrajNL = hcat(sol.u...);
xpert = xTrajNL .- xbar; #K*xpert[ix];
uNL = K*xpert[ix,:]; 

x0 = [0,110,0,0];
xTraj = hcat([exp(Acl*t)*x0 for t in tGrid]...);

xpNL = [begin 
        plot(sol.t,sol[ix[i],:].-xbar[ix[i]],label="NLin",lw=2,xlabel="Time(s)",ylabel=states[ix[i]]); 
        plot!(tGrid,xTraj[i,:],label="Lin",lw=2);
     end for i in 1:ns];

upNL = [begin 
     plot(sol.t,uNL[i,:],label="NLin",lw=2,xlabel="Time(s)",ylabel=controls[iu[i]]); 
     plot!(tGrid,uTraj[i,:],label="Lin",lw=2);
  end for i in 1:nu];

display(plot(xpNL...,upNL...));

UndefVarError: UndefVarError: xbar not defined

We see that the elevator angle violates magnitude limit. Therefore, this initial condition is not stabilizable with the designed control law.

# Challenge Problem Statement
Can we quantify the set in the $(\theta,V,\alpha,q)$ space such that all initial conditions are stabilized by the LQR controller, while satisfying limits on control and state (aerodynamic data)?
* Account for model uncertainty, process noise, etc.